# Accessing CDS services with Python 
## ISM*@ST on 24/08/20

Katharina Lutz

## Before we get started
### More resources 
This is a collection of links, where you can find more information:
 - Relevant Python Packages:
     - [`astroquery`](https://astroquery.readthedocs.io/en/latest/)
     - [`pyVO`](https://pyvo.readthedocs.io/en/latest/#)
     - [`MOCpy`](https://cds-astro.github.io/mocpy/)
     - [`ipyaladin`](https://github.com/cds-astro/ipyaladin)
 - AstroBetter posts on: [ipyaladin](https://www.astrobetter.com/blog/2020/05/04/the-cds-and-python-i-explore-the-sky-with-ipyaladin/), [MOCs](https://www.astrobetter.com/blog/2020/06/01/the-cds-and-python-ii-to-cover-or-not-to-cover-mocs-to-the-rescue/), [VizieR & XMatch](https://www.astrobetter.com/blog/2020/06/29/the-cds-and-python-iii-vizier-xmatch-20k-catalogues-and-tables-at-your-fingertips/) and [SIMBAD](https://www.astrobetter.com/blog/2020/07/06/the-cds-and-python-iv-simbad-the-yellow-pages-of-astronomical-sources/)
 - [Classic VO tutorials](http://www.euro-vo.org/index25a9.html?q=science/scientific-tutorials) translated to [Python](https://github.com/cds-astro/tutorials) (in progress)
 
### Importing packages

In [ ]:
from astropy.table import Table
from astropy.io import fits
from astropy import coordinates
from astropy.wcs import WCS
import astropy.units as u
import astropy.visualization as ap_vis

from astroquery.vizier import Vizier
from astroquery.cds import cds
from astroquery.simbad import Simbad

import pyvo

import ipyaladin.aladin_widget as ipyal
import mocpy
from urllib.parse import quote

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns 

## Overview of CDS services

 - <img src="Images/logo_simbad.png" alt="SIMBAD" style="width:40px; display: inline-block;"/> **SIMBAD**: <br>
 a bibliographic database containing interesting astronomical objects that have been studied in the literature. Look here for information on single objects, where are they located, what type of object is it, which paper mentions this object? <br>
 $\rightarrow$ **`astroquery.simbad`** and **`pyVO`**
 - <img src="Images/logo_vizier.png" alt="VizieR" style="width:40px; display: inline-block;"/> **VizieR**: <br>
 a database of catalogues and tables and their associated data. VizieR hosts both reference catalogues from large surveys at all wavelengths (2MASS, GAIA, NVSS, ...) and tables from papers. In addition data (images or spectra) associated with some tables are also provided. <br>
 $\rightarrow$ **`astroquery.vizier`** and **`pyVO`**
 - <img src="Images/xmatch2.png" alt="XMatch" style="width:40px; display: inline-block;"/> **XMatch**: <br>
 quick spatial cross matches between tables. These could be your own tables, any VizieR table or all of SIMBAD. <br>
 $\rightarrow$ **`astroquery.xmatch`**
 - <img src="Images/logo_aladin.png" alt="Aladin" style="width:40px; display: inline-block;"/> **Aladin & AladinLite**: <br>
 simple and fast acces to all large imaging surveys, which are provided in the form of HiPS.  <br>
 $\rightarrow$ **`ipyaladin`** (in development **`hips2fits`**)
 - **MOC**: <br>
 coverage maps, which allow for description of arbitrary patches on the sky, and quick unions or intersections inbetween these patches.  <br>
 $\rightarrow$  **`MOCpy`** and **`astroquery.cds`**

## Cautionary note before we get started:
Running queries in a loop might make our servers feel like you are attempting a DDoS attack ;) The `query_region` functions in the `astroquery` modules usually take lists of coordinates as input. An other option is to add a little `time.sleep(3)` in your loop, which makes the script wait between each step. 

## Science setting for today
We are looking for galaxies sufficient auxiliary data to study the distribution of the dust to atomic gas ratio. 

The steps we are going to take are the following:
 - Find all those galaxies that are within the observing footprints of GALEX, DES or SDSS, WISE and Herschel.
 - Quickly visualise these galaxies and their location on the sky
 - Pick one of these galaxies and get tabular data from VizieR for this galaxy
 - Pick one of these galaxies and search the associated data in the VizieR for HI data
 - Compile a list of bibliographic references for this galaxy

## Get coverage maps and compute their interesection
First we get the multiorder coverage (MOC) maps of different surveys. These MOCs can be used to describe any arbitrary patch on the sky. You could actually create MOCs just based on [polygons](https://cds-astro.github.io/mocpy/stubs/mocpy.MOC.html#mocpy.MOC.from_polygon) or your favourite [set of images](https://cds-astro.github.io/mocpy/stubs/mocpy.MOC.html#mocpy.MOC.from_fits_images), but here we are interested in the footprints of particular surveys or observations. These MOCs are stored on the CDS MOCServer and can be obtained with the `astroquery.cds` module. 

We know we want FUV imaging data from GALEX and F475W imaging data from HST, but we don't know what the data sets are called. So we first query the MOC server to get metadata for all data sets, which have IDs that contain some buzzwords. 

In [ ]:
info_galex = cds.find_datasets(meta_data="ID=*GALEX*")
info_herschel = cds.find_datasets(meta_data="ID=*HERSCHEL*")
info_wise = cds.find_datasets(meta_data="ID=*WISE*")
info_des = cds.find_datasets(meta_data="ID=*DES*")
info_sdss = cds.find_datasets(meta_data="ID=*SDSS*")

Generally, you get a lot of meta data for each of these datasets:

In [ ]:
info_galex

For the time being we are just happy to see the ID and and a descriptopn of the content

In [ ]:
info_galex['ID', 'obs_title']

In [ ]:
info_herschel['ID', 'obs_title']

In [ ]:
info_wise['ID', 'obs_title']

In [ ]:
info_des['ID', 'obs_title']

In [ ]:
info_sdss['ID', 'obs_title']

Cool, new we know more about these datasets, let's get their coverage map.

In [ ]:
moc_galex = cds.find_datasets(meta_data="ID=CDS/P/GALEXGR6/AIS/*UV", return_moc=True)
moc_sdss = cds.find_datasets(meta_data="ID=CDS/P/SDSS9/*", return_moc=True)
moc_wise = cds.find_datasets(meta_data="ID=CDS/P/allWISE/W*", return_moc=True)
moc_pacs = cds.find_datasets(meta_data="ID=ESAVO/P/HERSCHEL/PACS*", return_moc=True)
moc_spire = cds.find_datasets(meta_data="ID=ESAVO/P/HERSCHEL/SPIRE*", return_moc=True)
moc_des = cds.find_datasets(meta_data="ID=CDS/P/DES-DR1/*", return_moc=True)

In [ ]:
moc_dict = {'GALEX': moc_galex, 'SDSS': moc_sdss, 
            'WISE': moc_wise, 'PACS': moc_pacs, 'SPIRE': moc_spire, 
            'DES': moc_des}

And we can visualise these coverage maps:

In [ ]:
for survey in moc_dict.keys():
    fig = plt.figure(figsize=(6.3, 5.0))
    with mocpy.World2ScreenMPL(fig, 
             fov=200 * u.deg,
             center=coordinates.SkyCoord(0, 20, unit='deg', frame='icrs'),
             coordsys="icrs",
             rotation=coordinates.Angle(0, u.degree),
             projection="AIT") as wcs:
        ax = fig.add_axes([0.17, 0.17, 0.77, 0.77], projection=wcs)
        moc_dict[survey].fill(ax=ax, wcs=wcs, alpha=0.5, fill=True, color="crimson")
        ax.set_xlabel('RA')
        ax.set_ylabel('Dec')
        lon, lat = ax.coords[0], ax.coords[1]
        lon.set_major_formatter('hh:mm:ss')
        lat.set_major_formatter('dd:mm')
        lon.set_ticklabel(exclude_overlapping=True)
        lat.set_ticklabel(exclude_overlapping=True)
        lon.set_ticks(spacing=2 * u.hourangle)
        ax.coords.grid(color='black', linestyle='solid', alpha=0.5)
        ax.text(0.03, 0.03, survey, color='black', transform=ax.transAxes)

Now our idea was to get those regions on the sky that are either observed by SDSS or DES AND that are observed by all other surveys. To describe these regions with MOCs, we first build the union of SDSS and DES and then calculate the intersection between all remaining surveys

In [ ]:
moc_final = moc_sdss.union(moc_des)
for survey in moc_dict.keys():
    if survey in ['SDSS', 'DES']: 
        continue
    print('Forming intersection with: ', survey)
    moc_final = moc_final.intersection(moc_dict[survey])

and just a final look at the resulting shape:

In [ ]:
fig = plt.figure(figsize=(6.3, 5.0))
with mocpy.World2ScreenMPL(fig, 
         fov=200 * u.deg,
         center=coordinates.SkyCoord(0, 20, unit='deg', frame='icrs'),
         coordsys="icrs",
         rotation=coordinates.Angle(0, u.degree),
         projection="AIT") as wcs:
    ax = fig.add_axes([0.17, 0.17, 0.77, 0.77], projection=wcs)
    moc_final.fill(ax=ax, wcs=wcs, alpha=0.5, fill=True, color="darkcyan")
    ax.set_xlabel('RA')
    ax.set_ylabel('Dec')
    lon, lat = ax.coords[0], ax.coords[1]
    lon.set_major_formatter('hh:mm:ss')
    lat.set_major_formatter('dd:mm')
    lon.set_ticklabel(exclude_overlapping=True)
    lat.set_ticklabel(exclude_overlapping=True)
    lon.set_ticks(spacing=2 * u.hourangle)
    ax.coords.grid(color='black', linestyle='solid', alpha=0.5)
    ax.text(0.03, 0.03, 'Final MOC', color='black', transform=ax.transAxes)

Yep, that is a bit patchy, but this describes the regions on the sky we wanted and MOC is a great way to do this. 

## Get a catalogue from VizieR

In [ ]:
catalog_list_dustpedia = Vizier.find_catalogs('Dustpedia')
for k, v in catalog_list_dustpedia.items():
    print(k, ': ', v.description)

In [ ]:
vizir_dustpedia = Vizier(row_limit=-1)
results = vizir_dustpedia.get_catalogs('J/A+A/609/A37')
results

This is a list of `astropy` tables , which you can just use as any other `astropy` table.

In [ ]:
dustpedia_sample = results[0]
dustpedia_sample['Name', 'RAJ2000', 'DEJ2000']

Since positions of the galaxies in Dustpedia are given in the catalogue we can filter the catalogue to only show galaxies within the MOC. 

In [ ]:
indexes = moc_final.contains(dustpedia_sample['RAJ2000'].T * u.deg, 
                             dustpedia_sample['DEJ2000'].T * u.deg)
filtered_dustpedia = dustpedia_sample[indexes]
filtered_dustpedia

Alternatively we can query Vizier only within the MOC. (Currently this functionality is located within the MOCPy package but it might move over to astroquery at some point.)

In [ ]:
filtered_dustpedia_2 = moc_final.query_vizier_table('J/A+A/609/A37/sample', max_rows=100000)
filtered_dustpedia_2

Just a quick recap in between: we have seen, how we can
 - check the spatial coverage of different surveys. 
 - find intersections and unions of these coverage maps. 
 - find tables in VizieR.
 - download tables from VizieR.
 - filter tables to find those entries that are located within a MOC. 
 
## Visualise sources

One option is the AladinLite widget for Jupyter notebooks:

In [ ]:
aladin= ipyal.Aladin(survey='cds/P/DES-DR1/ColorIRG', target='NGC289', fov=0.5)
aladin

In [ ]:
aladin.add_table(filtered_dustpedia)

In [ ]:
aladin.target = 'M101'

Another option is to use HiPS, which Josh mentioned the other day. HiPS is a way of organising image, cube and catalogue data in a hierarchical way. The main principle is, the more you zoom in the more you see. All the surveys shown in the AladinLite widget above are organised as HiPS. Since all HiPS are saved in Healpix format getting cutouts is quite easy. There is a [webpage](http://alasky.u-strasbg.fr/hips-image-services/hips2fits), which has more information and a module in astroquery is under development. For the time being, we can use the following workaround:

In [ ]:
selected_hips = {'DES g': 'CDS/P/DES-DR1/g', 'GALEX FUV': 'CDS/P/GALEXGR6/AIS/FUV', 
                 'WISE W4': 'CDS/P/allWISE/W4', 'Spire 500mu': 'ESAVO/P/HERSCHEL/SPIRE-500', 
                 'PACS 100mu': 'ESAVO/P/HERSCHEL/PACS100'}
# Field of view in deg, here we want 7arcmin
fov = 7.0 / 60.
# Numbers of pixel with pixelsize of 0.6 arcsec
width = int(round(fov * 3600. / 0.6))
height = int(round(fov * 3600. / 0.6))
# Central coordinates 
sc = coordinates.SkyCoord(ra=13.17645, dec=-31.20581, unit=u.deg)
ra = sc.icrs.ra.deg
dec = sc.icrs.dec.deg

interval = ap_vis.AsymmetricPercentileInterval(1.0, 99.9)

for survey in selected_hips.keys():
    # get the image
    hips = selected_hips[survey]
    url = 'http://alasky.u-strasbg.fr/hips-image-services/hips2fits?' + \
          'hips={0}&width={1:d}&height={2:d}&fov={3:.4f}&'.format(quote(hips), width, height, fov)  + \
          'projection=TAN&coordsys=icrs&ra={0:.5f}&dec={1:.5f}'.format(ra, dec)
    ima = fits.open(url)
    
    # plot the image
    wcs = WCS(ima[0].header)
    fig = plt.figure(figsize=(6.3, 5.0))
    ax = fig.add_axes([0.17, 0.17, 0.77, 0.77], 
                      projection=wcs)
    vmin, vmax = interval.get_limits(ima[0].data)
    ax.imshow(ima[0].data, origin='lower',
              vmin=vmin, vmax=vmax, 
              cmap=mpl.cm.magma_r)
    # take care of axis labels ect.
    imsize = fov / (ima[0].header['CDELT2'])
    central_pix = wcs.wcs_world2pix(np.array([[ra], [dec]]).T, 0)
    ax.set_ylim(central_pix[0][1] - 0.5 * imsize,
                central_pix[0][1] + 0.5 * imsize)
    ax.set_xlim(central_pix[0][0] - 0.5 * imsize,
                central_pix[0][0] + 0.5 * imsize)
    lon, lat = ax.coords[0], ax.coords[1]
    lon.set_major_formatter('hh:mm:ss')
    lat.set_major_formatter('dd:mm')
    lon.set_ticklabel(exclude_overlapping=True)
    lat.set_ticklabel(exclude_overlapping=True)
    ax.set_xlabel('RA (J2000)')
    ax.set_ylabel('Dec (J2000)')
    ax.text(0.05, 0.05, '{}'.format(survey), color='black',
            transform=ax.transAxes)
    # save the image and close it
    ima.writeto('test_{}.fits'.format(survey), overwrite=True)
    ima.close()

This is still relavtively new technology and things are evolving. But for the time being I found it useful to visualise stars and HI in [150 galaxies](https://www.youtube.com/watch?v=aIWlsOf-mA0). 

## Accessing associated data in VizieR 
Associated data in VizieR are images, spectra or data cubes that are associated to a table in VizieR and are likely the basis of some of the measurement given in this table. If you use the VizieR web interface, a symbol will tell you whether a catalogue comes with associated data:

<img src="Images/vizier_assoc_data.png"/>

There is also a dedicated [webpage](http://cdsarc.u-strasbg.fr/assocdata/) to search through the associated data. However, here we want to use the power of pyVO to search for associated data for NGC289, the galaxy we already looked at above. 

Before we get into more details, let me take a minute to tell you a bit more about accessing tables in the framework of the Virtual Observatory: within the VO you can use the Table Access Protocol (TAP) to query tables. TAP is implemented in pyVO and you just need to give it the location of the server. Once the server is declared you can use ADQL, which is a flavour of SQL to query tables and databases which are located on the server. So let's see whether we can find associated data in the vicinity of NGC289.

First declare the server location.

In [ ]:
tap_vizier = pyvo.dal.TAPService('http://tapvizier.u-strasbg.fr/TAPVizieR/tap')

Then build the query. VizieR has a couple of databases you can query: 
 - there is a database of all tables in VizieR (`tables`), which you can use to find the right table.
 - each table can be queried (e.g. `J/A+A/609/A37/sample`). 
 - there is a database of all associated data (`"B/assocdata/obscore"`), which contains meta data for each image, spectrum, ect. This is the one we want to query here.

To better understand the table we are going to query, we'll just have a look at the first five rows:

In [ ]:
query_string = "SELECT TOP 5 * FROM \"B/assocdata/obscore\" "
top5obscore = tap_vizier.search(query_string)
top5obscore.to_table()

Again we get a lot of meta data, you can query for the location of the data (`RAJ2000` and `DEJ2000`) or the parent VizieR table for which we know the the name (`obs_collection`).
This time we just look for all data at radio wavelengths (between 0.1 and 1m in wavelengths), which were observed with the ATCA. 

In [ ]:
query_string = "SELECT * " + \
               "FROM \"B/assocdata/obscore\" " + \
               "WHERE em_min >= 0.01 AND em_max <= 1.0 AND facility_name='ATCA'"
ATCA_assoc_data = tap_vizier.search(query_string)
ATCA_assoc_data.to_table()

Now once more we can filter this list of potential HI observations by the MOC of complementary multi wavelength observations. 

In [ ]:
indexes = moc_final.contains(ATCA_assoc_data.to_table()['RAJ2000'].T * u.deg, 
                             ATCA_assoc_data.to_table()['DEJ2000'].T * u.deg)
filtered_assoc_data = ATCA_assoc_data.to_table()[indexes]
filtered_assoc_data

In [ ]:
assoc_data = fits.open('http://cdsarc.u-strasbg.fr/saadavizier/download?oid=864974549052030995')
assoc_data.info()

In [ ]:
wcs = WCS(assoc_data[0].header).celestial
fig = plt.figure(figsize=(6.3, 5.04))
ax = fig.add_axes([0.17, 0.17,
                   0.77, 0.77], projection=wcs)
ax.imshow(assoc_data[0].data[150, :, :], cmap='gist_gray_r')
ax.contour(assoc_data[0].data[150, :, :], colors='crimson', linewidths=0.5)
ax.set_xlabel('RAJ2000')
ax.set_ylabel('DEJ2000')

In [ ]:
spec = np.max(assoc_data[0].data, axis=(1,2))
fig = plt.figure(figsize=(6.3, 5.04))
ax = fig.add_axes([0.17, 0.17, 0.77, 0.77])
ax.plot(spec)

Et voila a beautiful HI double horn, which we can use for further analysis